In [1]:
import os

from cozepy import Coze, TokenAuth, COZE_CN_BASE_URL

COZE_API_TOKEN = "pat_lYvbkW8AhxKIGavZxgVGImzyC4XWAJBlcET9gEf463u1GJJDI3VDAqnIHYw1ozOj"

# 通过环境变量引入密钥，访问 coze.cn 服务
coze = Coze(auth=TokenAuth(COZE_API_TOKEN), base_url=COZE_CN_BASE_URL)


In [ ]:
import os  # noqa

# # Get an access_token through personal access token oroauth.
# coze_api_token = os.getenv("COZE_API_TOKEN")

from cozepy import Coze, TokenAuth, Message, ChatStatus, MessageContentType, ChatEventType  # noqa

# Init the Coze client through the access_token.
coze = Coze(auth=TokenAuth(token=COZE_API_TOKEN), base_url=COZE_CN_BASE_URL)

# Create a bot instance in Coze, copy the last number from the web link as the bot's ID.
bot_id = "7476363458222096419"
# The user id identifies the identity of a user. Developers can use a custom business ID
# or a random string.
user_id = "user id"

# Call the coze.chat.stream method to create a chat. The create method is a streaming
# chat and will return a Chat Iterator. Developers should iterate the iterator to get
# chat event and handle them.
for event in coze.chat.stream(
    bot_id=bot_id, user_id=user_id, additional_messages=[Message.build_user_question_text("How are you?")]
):
    if event.event == ChatEventType.CONVERSATION_MESSAGE_DELTA:
        message = event.message
        print(f"role={message.role}, content={message.content}")


In [6]:
import random
from cozepy import AudioFormat
import sys

def get_audio_filepath(coze):
    # if len(sys.argv) > 1:
    #     # 从程序运行参数中获取音频文件路径
    #     print(f"Get audio filepath from arg: {sys.argv[1]}")
    #     return sys.argv[1]

    # 随机选择一个语音
    voices = [i for i in coze.audio.voices.list()]
    voice = random.choice(voices)
    print(f"Get random voice: {voice.voice_id} {voice.name} from voices list of length: {len(voices)}")

    # 输入文本
    input_text = "你是基于什么大模型构建的"
    audio_path = os.path.join(os.path.expanduser("~"), f"coze_{voice.voice_id}_{input_text}.wav")

    # 使用语音合成API生成音频
    audio = coze.audio.speech.create(input=input_text, voice_id=voice.voice_id, response_format=AudioFormat.WAV)
    audio.write_to_file(audio_path)

    print(f"Get audio filepath from speech api: {audio_path}")
    return audio_path

audio_file_path = get_audio_filepath(coze)

Get random voice: 7468512265134981171 邻家小妹 from voices list of length: 130
Get audio filepath from speech api: /home/jeffye/coze_7468512265134981171_你是基于什么大模型构建的.wav


In [8]:
# 上传语音文件

from pathlib import Path

audio_file = coze.files.upload(file=Path(audio_file_path))

In [10]:
import base64
from cozepy import ChatEventType
from cozepy.util import write_pcm_to_wav_file
from cozepy import TokenAuth, Coze, ChatEventType, Message, MessageObjectString, AudioFormat

pcm_datas = b""
for event in coze.chat.stream(
    bot_id=bot_id,
    user_id=user_id,
    additional_messages=[
        Message.build_user_question_objects(
            [
                MessageObjectString.build_audio(file_id=audio_file.id),
            ]
        ),
    ],
):
    if event.event == ChatEventType.CONVERSATION_MESSAGE_DELTA:
        message = event.message
        print(event.message.content, end="", flush=True)
    elif event.event == ChatEventType.CONVERSATION_AUDIO_DELTA:
        pcm_datas += base64.b64decode(event.message.content)

# 将PCM音频数据保存为WAV文件
wav_audio_path = os.path.join(os.path.expanduser("~"), "coze_response_audio.wav")
write_pcm_to_wav_file(pcm_datas, wav_audio_path)
print(f"\nGet audio response from chat stream, save to {wav_audio_path}")

我是基于OpenAI的GPT-3.5模型构建的。
Get audio response from chat stream, save to /home/jeffye/coze_response_audio.wav


# audio streaming input

In [ ]:
import os
import tkinter as tk
from tkinter import ttk
from tkinter import scrolledtext
import pyaudio
import queue
import asyncio
import threading
from cozepy import AsyncCoze, TokenAuth
from cozepy.websockets.chat import AsyncWebsocketsChatClient, AsyncWebsocketsChatEventHandler
from cozepy.websockets.chat.events import ConversationAudioDeltaEvent, ConversationChatCompletedEvent, ConversationChatCanceledEvent
from cozepy.websockets.chat.events import ChatUpdateEvent, InputAudio
from cozepy.websockets.chat.events import InputAudioBufferAppendEvent

# 音频参数
RATE = 16000
CHANNELS = 1
FORMAT = pyaudio.paInt16
INPUT_BLOCK_TIME = 0.05
CHUNK = int(RATE * INPUT_BLOCK_TIME)


# Eino - coze的go后台

* https://www.cloudwego.io/zh/docs/eino/overview/bytedance_eino_practice/
* https://github.com/cloudwego/eino
* 用 Go 语言编写的终极大型语言模型（LLM）应用开发框架。它从开源社区中的诸多优秀 LLM 应用开发框架，如 LangChain 和 LlamaIndex 等获取灵感，同时借鉴前沿研究成果与实际应用，提供了一个强调简洁性、可扩展性、可靠性与有效性，且更符合 Go 语言编程惯例的 LLM 应用开发框架。